In [ ]:
import numpy as np
import pandas as pd


In [ ]:
def simulate_consumers(n=50000, seed=123):
    rng = np.random.default_rng(seed)

    mean = np.zeros(3)
    cov = np.array([
        [1.0, 0.4, 0.2],
        [0.4, 1.0, 0.3],
        [0.2, 0.3, 1.0]
    ])

    tastes = rng.multivariate_normal(mean, cov, size=n)
    T1, T2, T3 = tastes[:,0], tastes[:,1], tastes[:,2]

    X = rng.normal(0, 1, size=n)

    return pd.DataFrame({"T1":T1, "T2":T2, "T3":T3, "X":X})


In [ ]:
def compute_choice(df, prices, design_scores, betas=(1.0,1.0,1.0)):
    rng = np.random.default_rng(999)
    eps = rng.normal(0, 1, size=(len(df), 3))

    U1 = betas[0]*df["X"] + design_scores[0]*df["T1"] - prices[0] + eps[:,0]
    U2 = betas[1]*df["X"] + design_scores[1]*df["T2"] - prices[1] + eps[:,1]
    U3 = betas[2]*df["X"] + design_scores[2]*df["T3"] - prices[2] + eps[:,2]

    U = np.vstack([U1, U2, U3]).T
    choice = np.argmax(U, axis=1)

    return choice


In [ ]:
def run_policy_scenario(df, prices_before, design_before, prices_after, design_after):
    choice_before = compute_choice(df, prices_before, design_before)
    share_before = np.bincount(choice_before, minlength=3) / len(df)

    choice_after = compute_choice(df, prices_after, design_after)
    share_after = np.bincount(choice_after, minlength=3) / len(df)

    return share_before, share_after


In [ ]:
df = simulate_consumers()

prices_before = [20, 25, 30]
design_before = [1.0, 0.8, 0.6]

prices_after = [18, 25, 30]
design_after = [1.4, 0.8, 0.6]

share_before, share_after = run_policy_scenario(
    df,
    prices_before, design_before,
    prices_after, design_after
)

print("=== BASELINE SALES SHARES ===")
print(f"Product 1: {share_before[0]:.3f}")
print(f"Product 2: {share_before[1]:.3f}")
print(f"Product 3: {share_before[2]:.3f}\n")

print("=== COUNTERFACTUAL SALES SHARES ===")
print(f"Product 1: {share_after[0]:.3f}")
print(f"Product 2: {share_after[1]:.3f}")
print(f"Product 3: {share_after[2]:.3f}\n")

print("=== IMPACT OF CATALOG CHANGE ===")
print(f"Change in Product 1 share: {share_after[0] - share_before[0]:.3f}")
print(f"Change in Product 2 share: {share_after[1] - share_before[1]:.3f}")
print(f"Change in Product 3 share: {share_after[2] - share_before[2]:.3f}")
